# はじめに

Ansibleを使ってJupyterhub をLinuxホストに構築をする手順です。

## 手順を最新に更新する

以下のコマンドで master ブランチから最新の手順を取得します。もし手順を変更して未コミットの場合は、更新に失敗します。

In [ ]:
git status | grep "nothing to commit, working tree clean" \
 && git checkout master && git pull origin master \
 && git branch | grep -v master | xargs --no-run-if-empty git branch -d

更新に成功したら、F5キーなどで本ページをリロードさせます。

## 手順を修正して最新に適用させる

手順を修正したら、以下のコマンドで作業用のリモートブランチにコミットします。


In [ ]:
memo="up"
([ "$(git symbolic-ref --short HEAD)" == "master" ] && git checkout -b $(date +'%Y%m%d')-$(whoami) || true) \
 && git add . && (printf "$memo" | git commit -F-) && git push origin "$(git symbolic-ref --short HEAD)" --force

コミットに成功したら GitLab にログインして master ブランチに反映するための `Merge Request` を行います。
すでに手順に不備を指摘されて再びコミットする場合は `Merge request` は不要です。

## 手順の特殊な操作

### 変更中のファイル

In [ ]:
# 変更したファイル名一覧を取得
git status

In [ ]:
# 変更したファイル差分を取得
git diff

### コミット履歴

In [ ]:
# 簡易なコミット履歴（変更者とコメントのコミットツリーのみ）
git log --pretty=format:"%h - %an, %ar : %s"

In [ ]:
# 詳細なコミット履歴（ファイル差分あり）
git log -U1 --word-diff --color

### リビジョン番号を指定してプル

＜注＞最新でない手順を使ってのコマンド実行は推奨していません。

上記のコマンドで、手順を最新にしてから、rev変数の`リビジョン番号`を編集して、コマンドを実行してください。

In [ ]:
rev="リビジョン番号"
git checkout -b $(date +'%Y%m%d')-$(whoami) "$rev"



# 構築手順

## 注意

- SELinux は無効にしてください。CentOS はデフォルトで有効になっているので注意してください。
- ファイヤーウォールに注意してください。標準は 8000 ポートになっているので、CentOS などでは拒否されています。

## inventory ファイル

inventory ファイルの `jupyterhub` に対象ホストのIPアドレスを指定します。

In [ ]:
cat hosts

## 公開鍵の登録

対象ホストに以下の公開鍵を登録します。

In [ ]:
cat ~/.ssh/id_rsa.pub

登録が正しくできていれば、以下のコマンドの結果に`pong`が出力されます。

In [ ]:
USER=osboxes
ansible -i hosts -b -u "${USER}" -m ping jupyterhub

## Ansible で構築

以下のコマンドで Jupyterhub を構築します。

In [ ]:
ansible-playbook -i hosts -u "${USER}" site.yml

### bootstrap.sh

Jupyterhub から Jupyter プロセスを初めて起動させたときに `/etc/jupyterhub/bootstra.sh`が動くようになっています。
このスクリプトにより、一般権限のユーザで、bash_kernel をインストールしたり、各種extensionの有効化を行ったり、Jupyter を利用しやすいように初期設定を行っています。


In [ ]:
cat roles/jupyterhub/files/bootstrap.sh